In [54]:
# Load packages
# Established packages
import os
import numpy as np
import pandas as pd
import geopandas as gp
import datetime
import math
import xarray as xr # labels in the form of dimensions, coordinates and attributes
import matplotlib.pyplot as plt
import zarr

# Custom package covid19model
from covid19model.models import models
from covid19model.data import model_parameters, mobility #, sciensano, google
from covid19model.visualization.output import population_status, infected, show_map, show_graphs
from covid19model.models.time_dependant_parameter_fncs import lockdown_func, mobility_update_func, policies_wave1_4prev

# Moving average for visualisation
from covid19model.visualization.utils import moving_avg
from covid19model.models.utils import read_pops, read_coordinates_nis, initial_state

# Download function for complete calibration
# covid19model.optimization.run_optimization import full_calibration_wave1, full_calibration_wave2


# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
# This may be useful because the `covid19model` package is under construction
%autoreload 2

# Age stratification
stratification = 'phases'
# Spatial stratification
agg = 'arr'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = \
    model_parameters.get_interaction_matrices(stratification=stratification, spatial=agg)

In [63]:
# Check shape of initN
Nc_total.shape

(10, 10)

In [62]:
# Check if the contact matrices sum up to (approximately) the correct values:
value = abs(((Nc_home + Nc_work + Nc_schools + Nc_transport + Nc_leisure + Nc_others) - Nc_total).sum())
print(f"This value should be zero or very small: {value}")

This value should be zero or very small: 4.3357678558564317e-14


In [72]:
param_dict = model_parameters.get_COVID19_SEIRD_parameters(spatial='prov', stratification='phases', vaccination=False, intensity='all')

param_dict

{'Nc': array([[ 8.61114956,  0.83393502,  0.56828012,  1.7558695 ,  2.11664332,
          1.07760315,  0.93044761,  0.3881439 ,  0.19455493,  0.02083997],
        [ 1.62464949, 10.202451  ,  2.35316446,  3.11148901,  2.15810763,
          1.83025707,  0.38144829,  0.34986797,  0.11154411,  0.09183087],
        [ 0.90480164,  1.92316139,  6.15189471,  3.12873167,  2.21255394,
          3.27484973,  0.84221058,  0.33679633,  0.15444674,  0.10279999],
        [ 1.83312729,  1.66740536,  2.0515296 ,  5.27218355,  4.17086695,
          4.43751606,  2.2350914 ,  0.52389886,  0.31051015,  0.20396881],
        [ 2.0506185 ,  1.07320534,  1.34629471,  3.87046495,  5.38623365,
          4.03246549,  2.18372574,  0.9693051 ,  0.45428855,  0.16407803],
        [ 1.00280449,  0.87426288,  1.91407025,  3.95545971,  3.87338694,
          5.24028779,  2.52785647,  1.22236571,  0.76377765,  0.14512842],
        [ 1.02952013,  0.21664622,  0.58529214,  2.36885139,  2.49404232,
          3.00564658,  2.5